In [1]:
import gym
import numpy as np
import os

import ray.utils

from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.evaluation.sample_batch_builder import SampleBatchBuilder
from ray.rllib.offline.json_writer import JsonWriter

import minerl
import register_env

/projectnb/saenkog/juliusf/envs/ray/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
env_name = 'MineRLObtainDiamondDenseVectorObf-v0'

In [3]:
%env MINERL_DATA_ROOT=/projectnb/saenkog/juliusf/minerl

env: MINERL_DATA_ROOT=/projectnb/saenkog/juliusf/minerl


In [4]:
env = gym.make(env_name)
env = register_env.MineRLObservationWrapper(register_env.MineRLActionWrapper(env))

In [5]:
batch_builder = SampleBatchBuilder()
assert 'MINERL_DATA_ROOT' in os.environ
writer = JsonWriter(os.path.join(os.environ['MINERL_DATA_ROOT'], 'rllib', env_name))
prep = get_preprocessor(env.observation_space)(env.observation_space)
print("The preprocessor is", prep)

The preprocessor is <ray.rllib.models.preprocessors.TupleFlatteningPreprocessor object at 0x2b2b3851f7f0>


In [6]:
env.close()

In [7]:
data = minerl.data.make(env_name, data_dir=os.environ['MINERL_DATA_ROOT'])

In [9]:
for trajectory_name in data.get_trajectory_names():
    t = 0
    prev_action = None
    prev_reward = 0
    done = False
    obs = None
    info = None
    for obs, action, reward, next_obs, done in data.load_data(trajectory_name):
        obs = (obs['pov'], obs['vector'])
        next_obs = (next_obs['pov'], next_obs['vector'])
        action = action['vector']
        if prev_action is None:
            prev_action = np.zeros_like(action)
        
        batch_builder.add_values(
            t=t,
            eps_id=trajectory_name,
            agent_index=0,
            obs=prep.transform(obs),
            actions=action,
            action_prob=1.0,  # put the true action probability here
            rewards=reward,
            prev_actions=prev_action,
            prev_rewards=prev_reward,
            dones=done,
            infos=info,
            new_obs=prep.transform(next_obs))
        prev_action = action
        prev_reward = reward
        t += 1
    writer.write(batch_builder.build_and_reset())

100%|██████████| 16430/16430 [00:01<00:00, 8522.79it/s]
